In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# For reproducibility
np.random.seed(42)

In [ ]:
DATA_DIR = "data/random-images-for-face-emotion-recognition"

emotions = sorted(os.listdir(DATA_DIR))
print("Emotions:", emotions)

filepaths = []
labels = []
for idx, emo in enumerate(emotions):
    emo_dir = os.path.join(DATA_DIR, emo)
    for img_path in glob(os.path.join(emo_dir, "*.png")):
        filepaths.append(img_path)
        labels.append(idx)

print(f"Found {len(filepaths)} images")

Emotions: ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutrality', 'sadness', 'surprise']
Found 5558 images


In [3]:
# Pre-allocate arrays
N = len(filepaths)
X = np.zeros((N, 224, 224, 1), dtype="float32")
y = np.array(labels, dtype="int32")

# Load and normalize
for i, fp in enumerate(filepaths):
    img = load_img(fp, color_mode="grayscale", target_size=(224,224))
    arr = img_to_array(img) / 255.0
    X[i,:,:,0] = arr[:,:,0]

# One-hot encode labels
y_cat = to_categorical(y, num_classes=len(emotions))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, stratify=y, random_state=42
)
print("Train:", X_train.shape, y_train.shape)
print("Test: ", X_test.shape,  y_test.shape)

Train: (4446, 224, 224, 1) (4446, 8)
Test:  (1112, 224, 224, 1) (1112, 8)


In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(224,224,1)),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(64, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(128, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),
    
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(emotions), activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

c:\Users\Aiden\OneDrive\Current\Desktop\Multimodal-Emotion-Recognition\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,312 (42.61 MB)

 Trainable params: 11,169,864 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=32
)

Epoch 1/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 113s 872ms/step - accuracy: 0.2283 - loss: 2.9605 - val_accuracy: 0.1708 - val_loss: 10.8777
Epoch 2/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 113s 895ms/step - accuracy: 0.2922 - loss: 1.8611 - val_accuracy: 0.1708 - val_loss: 10.8077
Epoch 3/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 106s 838ms/step - accuracy: 0.3346 - loss: 1.7955 - val_accuracy: 0.2247 - val_loss: 7.1771
Epoch 4/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 110s 871ms/step - accuracy: 0.3569 - loss: 1.6874 - val_accuracy: 0.3303 - val_loss: 2.4554
Epoch 5/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 102s 807ms/step - accuracy: 0.3931 - loss: 1.5950 - val_accuracy: 0.3708 - val_loss: 1.8213
Epoch 6/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 112s 889ms/step - accuracy: 0.3987 - loss: 1.5914 - val_accuracy: 0.3685 - val_loss: 1.7920
Epoch 7/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 107s 845ms/step - accuracy: 0.4304 - loss: 1.4862 - val_accuracy: 0.4022 - val_loss: 1.7385
Epoch 8/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 139s 822ms/step - accuracy: 0.4686

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4%}")

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"],   label="val loss")
plt.legend(); plt.title("Loss")

plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="train acc")
plt.plot(history.history["val_accuracy"], label="val acc")
plt.legend(); plt.title("Accuracy")

plt.show()

In [ ]:
model.save("Models/face_emotion_cnn.h5")
print("Model saved to face_emotion_cnn.h5")